### Problem 2.
Estimating an economic model via ML.

In [1]:
import numpy as np
import scipy.optimize as opt
import scipy.stats as sts

In [2]:
c, k, w, r = np.loadtxt(r'C:\Users\Max\Documents\BootCamp2017\Econ\Wk4_StrEst\data\MacroSeries.txt', delimiter=',').T

In [3]:
def get_z_from_3(alpha, mu, k, w):
    
    z =  np.log(w/((1 - alpha) * k ** alpha))
    z_lag1 = np.concatenate([np.array([mu]), z[:-1]])
    
    return z, z_lag1

def norm_loglik(params, *args):
    alpha, sigma, rho, mu = params
    k, w = args
    
    z, z_lag = get_z_from_3(alpha, mu, k, w)
    z_mean = (rho * z_lag) + (1 - rho) * mu
    
    loglik = sts.norm(loc=z_mean, scale=sigma).logpdf(z).sum()
    
    return -1*loglik

In [4]:
alpha_0 = 0.7
sigma_0 = 0.7
rho_0 = 0.1
mu_0 = 4
params_init = (alpha_0, sigma_0, rho_0, mu_0)
args_a = (k, w)

In [5]:
res_a = opt.minimize(norm_loglik, params_init, args=args_a,
                   method='L-BFGS-B',
                   bounds=((1e-10, 1-1e-10), (1e-10, None), 
                          (1e-10-1, 1-1e-10), (1e-10, 30))
                    )

alpha, sigma, rho, mu = res_a.x
ll = -1*res_a.fun

print('Estimation results',
      '\n\nalpha:\t\t\t', alpha,
      '\nsigma:\t\t\t', sigma,
      '\nrho:\t\t\t', rho,
      '\nmu :\t\t\t', mu,
     '\nLogLikelikhood :\t', ll)

print('\nHessian Matrix:\n', res_a.hess_inv.todense())
res_a


Estimation results 

alpha:			 0.914712683312 
sigma:			 1.00509146545 
rho:			 0.142783922391 
mu :			 4.02449563815 
LogLikelikhood :	 -93.3694967498

Hessian Matrix:
 [[ 0.08084526 -0.22017763 -0.19433938 -0.06372927]
 [-0.22017763  1.37852952 -0.01450835 -0.03417128]
 [-0.19433938 -0.01450835  0.95911674 -0.01293341]
 [-0.06372927 -0.03417128 -0.01293341  0.99468967]]


C:\Users\Max\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: invalid value encountered in log
  app.launch_new_instance()
C:\Users\Max\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
C:\Users\Max\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


      fun: 93.369496749844799
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-35.47848166,  97.56765849,  -1.3340113 ,  -8.7381494 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 30
      nit: 3
   status: 0
  success: True
        x: array([ 0.91471268,  1.00509147,  0.14278392,  4.02449564])

In [6]:
# trying to find some better initial guesses here...

alphas = np.linspace(1e-10, 1-1e-10, 5)
sigmas = np.logspace(1e-10, 100, 5)
rhos = np.linspace(1e-10-1, 1-1e-10, 5)
mus = np.logspace(1e-10, 30, 5)

init_opt = np.array([0, 1, 2, 3])
fun_opt = 10000
for a in alphas:
    for s in sigmas:
        for r in rhos:
            for m in rhos:
                params_iter = np.array([a, s, r, m])
                
                res_iter = opt.minimize(norm_loglik, params_iter,
                                        args=args_a,
                   method='L-BFGS-B',
                   bounds=((1e-10, 1-1e-10), (1e-10, None), 
                          (1e-10-1, 1-1e-10), (1e-10, 30))
                    )
                this_fun = res_iter.fun
                if this_fun < fun_opt:
                    fun_opt = np.copy(this_fun)
                    init_opt = np.copy(params_iter)
                    
init_opt               

C:\Users\Max\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: RuntimeWarning: invalid value encountered in log
  app.launch_new_instance()
C:\Users\Max\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
C:\Users\Max\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)
C:\Users\Max\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1692: RuntimeWarning: invalid value encountered in greater
  cond0 = self._argcheck(*args) & (scale > 0)
C:\Users\Max\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1693: RuntimeWarning: invalid value encountered in greater
  cond1 = self._support_mask(x) & (scale > 0)


array([ 0.25,  1.  ,  0.  ,  1.  ])

In [7]:
res_opt = opt.minimize(norm_loglik, init_opt, args=args_a,
                   method='L-BFGS-B',
                   bounds=((1e-10, 1-1e-10), (1e-10, None), 
                          (1e-10-1, 1-1e-10), (1e-10, 30))
                    )

alpha_opt, sigma_opt, rho_opt, mu_opt = res_opt.x
ll_opt = -1*res_opt.fun

print('Estimation results',
      '\n\nalpha:\t\t\t', alpha_opt,
      '\nsigma:\t\t\t', sigma_opt,
      '\nrho:\t\t\t', rho_opt,
      '\nmu :\t\t\t', mu_opt,
     '\nLogLikelikhood :\t', ll_opt)

print('\n Inverse Hessian Matrix:\n', res_opt.hess_inv.todense())

res_opt

Estimation results 

alpha:			 0.457518997955 
sigma:			 0.091996300808 
rho:			 0.720488273392 
mu :			 9.52268275846 
LogLikelikhood :	 96.7069080734

 Inverse Hessian Matrix:
 [[  1.36153562e+01  -1.94022855e+00  -1.13673387e+01  -1.84879761e+02]
 [ -1.94022855e+00   3.11710842e-01   1.74121115e+00   2.62383448e+01]
 [ -1.13673387e+01   1.74121115e+00   9.92181361e+00   1.53983802e+02]
 [ -1.84879761e+02   2.62383448e+01   1.53983802e+02   2.51077064e+03]]


      fun: -96.706908073365724
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00937916,  0.0016442 ,  0.00018758,  0.00066791])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 315
      nit: 43
   status: 0
  success: True
        x: array([ 0.457519  ,  0.0919963 ,  0.72048827,  9.52268276])

I think this result is at least a little more reasonable, however I have had my problems with this problem set...  For this reason consider the following exercise:

## Second approach

A shame that i did all of this before I caught the following though. We also talked about his with Justin a little.
    
I do not really understand the statement about the distribution of $z_{t}$. How can there be $z_{t-1}$ in the definition of the mean? Is this supposed to be some conditional one-step-ahead kind of distribution? However, this would give me a different mean when setting up my log liklihood and would violate the iid assumptions needed to even being able to set up the likelihood. (I noticed that after getting really weird values for thee loglikelihood, namly positive ones, like above. Hower I found out that this can happen. I was only used to seing negative values for the log logliklihood).
Since $z$ is an ar-process its unconditinal mean is just $\mu$ and its unconditional variance is $\frac{\sigma^2}{1-\rho^2}$

In [8]:
# back out alha from equations 3 and 4
alpha_vec = 1/(w/(r*k)+1)
alpha = np.mean(alpha_vec)
alpha

0.41731988399318171

In [9]:
def get_z3(alpha, k, w):
    
    z =  np.log(w/((1 - alpha) * k ** alpha))
    
    return z

In [10]:
zi = get_z3(alpha, k, w)

In [11]:
# getting a good guess of mu
mu_0 = np.mean(zi)
mu_0

10.083681502893953

In [12]:
def norm_loglik2(params, *args):
    sigma, rho, mu = params
    k, w, zi, alpha = args
    
    std_of_zi = sigma/np.sqrt(1 - rho **2)
    
    loglik = sts.norm.logpdf(zi, loc=mu, scale=std_of_zi).sum()
    
    return -1*loglik

In [13]:
# some initial values
sigma_0 = 12
rho_0 = 0.1
params_init2 = (sigma_0, rho_0, mu_0)
args_a = (k, w, zi, alpha)

In [14]:
norm_loglik2(params_init2, k, w, zi, alpha)

340.89352991967343

In [15]:
# I restrit rho to be between 0 and 1 since an a-persistent shock
# is usually not what economist bouild into their models

res_a2 = opt.minimize(norm_loglik2, params_init2, args=args_a,
                   method='L-BFGS-B',
                   bounds=((1e-10, None), 
                          (0, 1-1e-10), (1e-10, 30))
                    )

sigma2, rho2, mu2 = res_a2.x
ll2 = -1*res_a2.fun

print('Estimation results',
      '\n\nalpha (not estimated):\t', alpha,
      '\nsigma:\t\t\t', sigma2,
      '\nrho:\t\t\t', rho2,
      '\nmu :\t\t\t', mu2,
     '\nLogLikelikhood :\t', ll2)

print('\nInverse Hessian Matrix:\n', res_a2.hess_inv.todense())
res_a2

Estimation results 

alpha (not estimated):	 0.417319883993 
sigma:			 0.137455358099 
rho:			 1.78364862783e-10 
mu :			 10.0836817708 
LogLikelikhood :	 56.551572805

Inverse Hessian Matrix:
 [[  1.39971301e-01  -1.56068437e-05  -1.55608049e-01]
 [ -1.56068437e-05   9.99999999e-01  -4.37566333e-05]
 [ -1.55608049e-01  -4.37566333e-05   1.73297438e-01]]


      fun: -56.551572804965993
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00259774,  0.        ,  0.00144382])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 60
      nit: 6
   status: 0
  success: True
        x: array([  1.37455358e-01,   1.78364863e-10,   1.00836818e+01])

everything but rho seems to be quite stabe regradless of the initial conditions. However, I just thought I coula also rformulate everything in terms of te iid disturbane to z. This is probably the cleanest way of doing this:

In [16]:
def norm_eps(params, *args):
    sigma, rho, mu = params
    zi = args
    
    zi_1 = np.concatenate([np.array([mu]), zi[:-1]])
    
    eps = zi - rho*zi_1 -(1 - rho) * mu
    
    loglik = sts.norm.logpdf(eps, loc=0, scale=sigma).sum()
    
    return -1*loglik

In [17]:
sigma_0 = 12
rho_0 = -0.9
params_init_eps = [sigma_0, rho_0, mu_0]

In [18]:
res_aeps = opt.minimize(norm_eps, params_init_eps, args=(zi),
                   method='L-BFGS-B',
                   bounds=((1e-10, None), 
                          (1e-10-1, 1-1e-10), (1e-10, 30))
                    )

sigma_aeps, rho_aeps, mu_aeps = res_aeps.x
ll_aeps = -1*res_aeps.fun

print('Estimation results',
      '\n\nalpha (not estimated):\t', alpha,
      '\nsigma:\t\t\t', sigma_aeps,
      '\nrho:\t\t\t', rho_aeps,
      '\nmu :\t\t\t', mu_aeps,
     '\nLogLikelikhood :\t', ll_aeps)

print('\nInverse Hessian Matrix:\n', res_aeps.hess_inv.todense())
res_aeps

Estimation results 

alpha (not estimated):	 0.417319883993 
sigma:			 0.137455359372 
rho:			 -0.900000001937 
mu :			 10.0836817806 
LogLikelikhood :	 56.551572805

Inverse Hessian Matrix:
 [[  1.40073434e-01  -1.47359866e-04  -1.55425782e-01]
 [ -1.47359866e-04   9.99999724e-01  -5.27789039e-04]
 [ -1.55425782e-01  -5.27789039e-04   1.72766374e-01]]


      fun: -56.551572804955129
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -2.58353339e-03,  -2.34479103e-05,   1.47224455e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 60
      nit: 6
   status: 0
  success: True
        x: array([  0.13745536,  -0.9       ,  10.08368178])

This gives the exact same result as the one above. INcludong the sensitivity of rho on the initial value.

## b)

In [19]:
def get_z_from_4(alpha, mu, k, r):
    
    z = np.log(r / (alpha * k ** (alpha - 1)))
    z_lag1 = np.concatenate([np.array([mu]), z[:-1]])
    
    return z, z_lag1
                
def norm_loglik_b(params, *args):
    alpha, sigma, rho, mu = params
    k, r = args
    
    z, z_lag = get_z_from_4(alpha, mu, k, r)
    z_mean = (rho * z_lag) + (1 - rho) * mu
    
    loglik = sts.norm(loc=z_mean, scale=sigma).logpdf(z).sum()
    
    return -1*loglik

In [20]:
norm_loglik_b(params_init, k, r)

156.4580235262986

In [21]:
args_b = (k, r)
other_inits = np.array([1,  0.1,  0.9,  7])

res_opt_b = opt.minimize(norm_loglik_b, init_opt, args=args_b,
                   method='L-BFGS-B',
                   bounds=((1e-10, 1-1e-10), (1e-10, None), 
                          (1e-10-1, 1-1e-10), (1e-10, 30))
                    )

alpha_opt_b, sigma_opt_b, rho_opt_b, mu_opt_b = res_opt_b.x
ll_opt_b = -1*res_opt_b.fun

print('Estimation results',
      '\n\nalpha:\t\t\t', alpha_opt_b,
      '\nsigma:\t\t\t', sigma_opt_b,
      '\nrho:\t\t\t', rho_opt_b,
      '\nmu :\t\t\t', mu_opt_b,
     '\nLogLikelikhood :\t', ll_opt_b)

print('\nInverse Hessian Matrix:\n', res_opt_b.hess_inv.todense())

res_opt_b

Estimation results 

alpha:			 0.9999999999 
sigma:			 1e-10 
rho:			 0.9999999999 
mu :			 1e-10 
LogLikelikhood :	 2099.18381261

Inverse Hessian Matrix:
 [[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


      fun: -2099.1838126068651
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.40779305e+14,   3.50015556e+10,  -2.08769961e+00,
         3.50999721e+11])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 365
      nit: 12
   status: 2
  success: False
        x: array([  1.00000000e+00,   1.00000000e-10,   1.00000000e+00,
         1.00000000e-10])

This doesn't look good. This approach is just not valid in MLE, correct? I can't just construct a log likelihood function out of serially correlated terms. The independence assumptions shich allows me to multiply the pdf's of the single observations is clearly not given in his case.

We can get $\alpha$ and have the good initial guess for $\mu$ in the same way we did above.

In [22]:
def get_z4(alpha, r, w):
    
    z = np.log(r / (alpha * k ** (alpha - 1)))
    
    return z

zi = get_z4(alpha, r, w)

# This looks good but a little different than earlier.
mu4 = np.mean(zi)
mu4

10.082870874830103

In [23]:
def norm_loglik4(params, *args):
    sigma, rho, mu = params
    zi, alpha = args
    
    std_of_zi = sigma/np.sqrt(1 - rho **2)
    
    loglik = sts.norm.logpdf(zi, loc=mu, scale=std_of_zi).sum()
    
    return -1*loglik

In [24]:
sigma_0 = 0.7
rho_0 = 0.4
params_initb2 = (sigma_0, rho_0, mu4)
args_b2 = (zi, alpha)

res_b2 = opt.minimize(norm_loglik4, params_initb2, args=args_b2,
                   method='L-BFGS-B',
                   bounds=((1e-10, None), 
                          (0, 1-1e-10), (1e-10, 30))
                    )

sigmab2, rhob2, mub2 = res_b2.x
llb2 = -1*res_b2.fun

print('Estimation results',
      '\n\nalpha (not estimated):\t', alpha,
      '\nsigma:\t\t\t', sigmab2,
      '\nrho:\t\t\t', rhob2,
      '\nmu :\t\t\t', mub2,
     '\nLogLikelikhood :\t', llb2)

print('\n Inverse Hessian Matrix:\n', res_b2.hess_inv.todense())
res_b2

Estimation results 

alpha (not estimated):	 0.417319883993 
sigma:			 0.119379231299 
rho:			 0.0725624871089 
mu :			 10.0828711265 
LogLikelikhood :	 70.3871858225

 Inverse Hessian Matrix:
 [[  1.22466360e-04  -4.86741312e-03   2.99544814e-03]
 [ -4.86741312e-03   5.59595191e-01  -2.79602493e-04]
 [  2.99544814e-03  -2.79602493e-04   9.99942238e-01]]


      fun: -70.387185822535841
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.56319402e-05,  -1.42108547e-06,   1.79056769e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 52
      nit: 5
   status: 0
  success: True
        x: array([  0.11937923,   0.07256249,  10.08287113])

This is reassuring beccause the results look very much like the results in part a). 

And once more just reformulating everything interms of epsilon:

In [25]:
sigma_0 = 15
rho_0 = -0.124
params_eps2 = [sigma_0, rho_0, 16]

res_beps = opt.minimize(norm_eps, params_eps2, args=(zi),
                   method='L-BFGS-B',
                   bounds=((1e-10, None), 
                          (1e-10-1, 1-1e-10), (1e-10, 30))
                    )

sigma_beps, rho_beps, mu_beps = res_beps.x
ll_beps = -1*res_beps.fun

print('Estimation results',
      '\n\nalpha (not estimated):\t', alpha,
      '\nsigma:\t\t\t', sigma_beps,
      '\nrho:\t\t\t', rho_beps,
      '\nmu :\t\t\t', mu_beps,
     '\nLogLikelikhood :\t', ll_beps)

print('\nInverse Hessian Matrix:\n', res_beps.hess_inv.todense())
res_beps

Estimation results 

alpha (not estimated):	 0.417319883993 
sigma:			 0.119694768829 
rho:			 -0.124000025949 
mu :			 10.0828708472 
LogLikelikhood :	 70.3871858228

Inverse Hessian Matrix:
 [[  7.12299286e-05   4.42192780e-07   1.12091145e-06]
 [  4.42192780e-07   1.00000000e+00  -1.10490966e-06]
 [  1.12091145e-06  -1.10490966e-06   1.40543044e-04]]


      fun: -70.387185822754219
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00010516,  0.        , -0.00015774])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 108
      nit: 10
   status: 0
  success: True
        x: array([  0.11969477,  -0.12400003,  10.08287085])

Same result here, even for crazy guesses sigma and mu come close to the values of 0.12 and 10. Only rho is not  estimated precicely at all.

## c)

calculate z_star for r=1 and k=7500000

In [26]:
z_star = np.log(1/(alpha * 7500000 **(alpha -1)))

In [27]:
def z_cdf(z_star, sigma, rho, mu):                             
    
    std_of_zi = sigma/np.sqrt(1 - rho **2)
    
    cdf_vals = sts.norm.cdf(z_star, loc=mu, scale=std_of_zi)
    
    return cdf_vals

In [28]:
1 - z_cdf(z_star, sigmab2, rhob2, mub2)

0.44981042052737463

The probability that r is higher than one in the next period is approximately 0.45.